In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from liquidity.response_functions.functional_form import rescaled_form, scaling_form
from liquidity.util.utils import bin_data_into_quantiles, smooth_outliers
from liquidity.response_functions.features import compute_aggregate_features
from liquidity.response_functions.fit import find_shape_parameters, compute_scale_factors, renormalise
from liquidity.response_functions.price_response_functions import compute_conditional_aggregate_impact

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr'

In [4]:
%load_ext autoreload
%autoreload 2

## Load data

In [5]:
# TSLA Example
current_dir = os.path.abspath('.')
root_dir = os.path.join(current_dir, '..', '..')
data_dir = os.path.join(root_dir, 'data', 'market_orders')
filename = "TSLA-2017-NEW.csv"
stock_file_path = os.path.join(data_dir, filename)
tsla_orderbook_states = pd.read_csv(stock_file_path)

In [8]:
tsla_orderbook_states

,Unnamed: 0,event_timestamp,sign,side,lob_action,order_executed,execution_price,size,ask,bid,...,bid_queue_size_mean,midprice_change,R1,spread,date,daily_R1,daily_vol,daily_num,norm_size,signed_volume
0,0,2017-01-03 10:30:19.351489154,1,ASK,UPDATE,True,219.37,1,219.37,219.28,...,135.963741,0.000,0.000,0.09,2017-01-03,0.021159,277786,3279,0.008043,1
1,1,2017-01-03 10:30:19.351537884,1,ASK,REMOVE,True,219.37,15,219.37,219.28,...,135.963741,0.005,0.005,0.09,2017-01-03,0.021159,277786,3279,0.120647,15
2,2,2017-01-03 10:30:19.351549791,1,ASK,UPDATE,True,219.38,80,219.38,219.28,...,135.963741,0.005,0.005,0.10,2017-01-03,0.021159,277786,3279,0.643452,80
3,3,2017-01-03 10:30:19.351601187,1,ASK,UPDATE,True,219.38,80,219.38,219.29,...,135.963741,0.000,0.000,0.09,2017-01-03,0.021159,277786,3279,0.643452,80
4,4,2017-01-03 10:30:19.351702603,1,ASK,REMOVE,True,219.38,40,219.38,219.29,...,135.963741,0.050,0.050,0.09,2017-01-03,0.021159,277786,3279,0.321726,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411991,3572,2017-12-29 14:59:11.533706465,1,ASK,REMOVE,True,311.06,321,311.06,311.02,...,151.020334,0.025,0.025,0.04,2017-12-29,0.019713,322243,3577,2.112474,321
1411992,3573,2017-12-29 14:59:50.386983509,-1,BID,REMOVE,True,311.03,200,311.10,311.03,...,151.020334,-0.030,0.030,0.07,2017-12-29,0.019713,322243,3577,1.324325,-200
1411993,3574,2017-12-29 14:59:50.387892067,-1,BID,UPDATE,True,311.02,200,311.05,311.02,...,151.020334,0.000,-0.000,0.03,2017-12-29,0.019713,322243,3577,1.324325,-200
1411994,3575,2017-12-29 14:59:50.387904347,-1,BID,UPDATE,True,311.02,2,311.05,311.02,...,151.020334,0.000,-0.000,0.03,2017-12-29,0.019713,322243,3577,0.013243,-2


## Finite-size scaling of conditional distrubutions (functions or curves)

#### Fitting functions

   Scaling function: $\mathscr{F}(x) = \frac{x}{(1 + |x|^\alpha)^{\frac{\beta}{\alpha}}}$
   Rescaled form:  $R(ΔV,T) \approx R(1)T^\chi \times  \mathscr{F}\left(\frac{ΔV}{V_{D}T^\varkappa}\right)$
   Scaling form: $R(ΔV) \approx R_T \times  \mathscr{F}\left(\frac{ΔV}{\mathcal{E}_T}\right)$


   In order to determine the rescaling exponents $\chi$ and $\varkappa$:

   1. The shape of $R(ΔV,T)$ is fitted for all T using the scaling form with scaling function given by $\mathscr{F}(x)$.

   2. Once shape parameters $\alpha$ and $\beta$ are fixed, we can map out scale factors $QN$ and $RN$, by fitting the
    rescaled_form on each $T$ whilst keeping constant the value of $\alpha$ and $\beta$ for all $T$.